In [1]:
with open("./input.txt", "r") as file:
    data = [
        (d, int(dist))
        for (d, dist) in [row.split(" ") for row in file.read().strip().split("\n")]
    ]

# Part 1

In [2]:
import easytree

In [3]:
def sign(x):
    """
    Returns the sign of x
    """
    return -1 if x < 0 else 1


def move(rope, direction):
    """
    Move a 2-knot rope 
    
    Parameters
    ----------
    rope : dict[str:dict[str,int]]
        The head and tail coordinates
    direction : str
        One of U, D, L and R
        
    Returns
    -------
    rope : dict[str:dict[str,int]]
        The new position of the rope
        
    Note
    ----
    The rope object is of the form 
    {
        "head":{
            "x":0, "y":0
        },
        "tail":{
            "x":0, "y":0
        }
    }
    """
    if direction not in "UDLR":
        raise ValueError(f"Expected direction to be one of U, D, L or R, received {direction}")
        
    # make a copy for immutability
    rope = easytree.Tree(rope)
    
    # move head of the rope
    if direction == "U":
        rope.head.y += 1
    elif direction == "D":
        rope.head.y -= 1
    elif direction == "R":
        rope.head.x += 1
    elif direction == "L":
        rope.head.x -= 1

    # determine the vertical and horizontal distance of the head and tail
    disty = rope.head.y - rope.tail.y
    distx = rope.head.x - rope.tail.x

    # if they are overlapping or touching,
    # then the squared (manhattan) distance is
    # less or equal to 2 blocks
    if (distx ** 2 + disty ** 2) <= 2:
        return rope
    
    if abs(distx) + abs(disty) > 3: 
        raise RuntimeError(
            f"""
            Expected distance deltas to be no greater than 2 each, 
            found dx={distx} and dy={distx}
            """
        )
    
    # if they are aligned, then one of the
    # distances must be 0 (and the other must be 2)
    if distx * disty == 0:
        if distx == 0:
            rope.tail.y += sign(disty)
        else:
            rope.tail.x += sign(distx)
            
        return rope
    
    # otherwise, move each by 1 in the
    # direction of the difference
    rope.tail.y += sign(disty)
    rope.tail.x += sign(distx)
    
    return rope

def solve(data):
    # initialize rope
    rope = easytree.Tree({
        "head":{
            "x":0, "y":0
        },
        "tail":{
            "x":0, "y":0
        }
    })
    
    # history of rope coordinates 
    history = [rope]
    
    for direction, distance in data: 
        for _ in range(distance):
            history.append(
                move(history[-1], direction)
            )
            
    return len(set([(r.tail.x, r.tail.y) for r in history]))

solve(data)

6098

# Part 2

In [4]:
def move(rope, direction):
    """
    Generalized version of the above
    
    Parameters
    ----------
    rope : list[dict[str,int]]
        List of knot coordinates
    direction : str
        One of U, D, L and R
        
    Returns
    -------
    rope : list[dict[str,int]]
        The new position of the rope
        
    Note
    ----
    The rope object is of the form 
    [
        {
            "x":0, "y":0
        },
        ...
        {
            "x":0, "y":0
        }
    ]
    """
    if direction not in "UDLR":
        raise ValueError(f"Expected direction to be one of U, D, L or R, received {direction}")
        
    # make a copy (to ensure immutability)
    rope = easytree.Tree(rope)
    
    # move head of the rope
    if direction == "U":
        rope[0].y += 1
    elif direction == "D":
        rope[0].y -= 1
    elif direction == "R":
        rope[0].x += 1
    elif direction == "L":
        rope[0].x -= 1

    for index in range(1, len(rope)):
        # determine the vertical and horizontal distance of the head and tail
        disty = rope[index-1].y - rope[index].y
        distx = rope[index-1].x - rope[index].x

        # if they are overlapping or touching,
        # then the squared (manhattan) distance is
        # less or equal to 2 blocks
        if (distx ** 2 + disty ** 2) <= 2:
            continue

        if abs(distx) + abs(disty) > 4: 
            raise RuntimeError(
                f"""
                Expected distance deltas to be no greater than 2 each, 
                found dx={distx} and dy={distx} between knot {index-1} 
                and knot {index} for rope {rope}
                """
            )

        # if they are aligned, then one of the
        # distances must be 0 (and the other must be ???)
        if distx * disty == 0:
            if distx == 0:
                rope[index].y += sign(disty)
            else:
                rope[index].x += sign(distx)

            continue

        # otherwise, move each by 1 in the
        # direction of the difference
        rope[index].y += sign(disty)
        rope[index].x += sign(distx)
    
    return rope

def solve(data):
    # initialize rope
    rope = easytree.Tree([
        {
            "x":0, "y":0
        }
        for _ in range(10)
    ])
    
    # history of rope coordinates 
    history = [rope]
    
    for i, (direction, distance) in enumerate(data): 
        for step in range(distance):
            history.append(
                move(history[-1], direction)
            )
            
    return history, len(set([(r[-1].x, r[-1].y) for r in history]))

history, count = solve(data)
count

2597

In [5]:
import easychart

chart = easychart.new("spline", zoom="xy")
chart.title = "Positions of head and tail of rope"
chart.plot([[r[0].x, r[0].y] for r in history], name="head")
chart.plot([[r[-1].x, r[-1].y] for r in history], name="tail")
chart

Tree({'series': [{'data': [[0, 0], [0, -1], [0, -2], [-1, -2], [-2, -2], [-2, -1], [-3, -1], [-3, 0], [-3, 1], [-2, 1], [-3, 1], [-3, 0], [-3, -1], [-3, 0], [-3, 1], [-2, 1], [-1, 1], [-1, 2], [-2, 2], [-3, 2], [-2, 2], [-2, 1], [-3, 1], [-4, 1], [-3, 1], [-3, 2], [-3, 3], [-4, 3], [-4, 4], [-4, 3], [-3, 3], [-3, 4], [-3, 5], [-3, 4], [-3, 3], [-4, 3], [-4, 2], [-4, 3], [-5, 3], [-5, 4], [-5, 3], [-5, 2], [-5, 3], [-6, 3], [-7, 3], [-7, 2], [-8, 2], [-8, 1], [-8, 0], [-8, 1], [-8, 0], [-8, -1], [-7, -1], [-8, -1], [-9, -1], [-8, -1], [-8, 0], [-7, 0], [-6, 0], [-6, 1], [-6, 2], [-6, 1], [-6, 0], [-5, 0], [-5, -1], [-4, -1], [-4, 0], [-4, 1], [-4, 0], [-4, 1], [-4, 2], [-4, 1], [-4, 0], [-5, 0], [-5, -1], [-5, -2], [-4, -2], [-3, -2], [-3, -1], [-4, -1], [-3, -1], [-3, 0], [-3, 1], [-3, 0], [-3, -1], [-2, -1], [-2, -2], [-2, -3], [-2, -2], [-3, -2], [-4, -2], [-3, -2], [-4, -2], [-5, -2], [-5, -3], [-5, -4], [-5, -3], [-5, -2], [-5, -3], [-5, -4], [-6, -4], [-7, -4], [-7, -3], [-8, -3], [-8, -4], [-8, -5], [-9, -5], [-8, -5], [-7, -5], [-7, -4], [-7, -3], [-7, -4], [-6, -4], [-6, -3], [-6, -2], [-6, -3], [-6, -4], [-6, -3], [-5, -3], [-4, -3], [-5, -3], [-4, -3], [-5, -3], [-5, -2], [-5, -1], [-5, -2], [-5, -3], [-6, -3], [-7, -3], [-7, -4], [-8, -4], [-9, -4], [-9, -3], [-9, -2], [-8, -2], [-7, -2], [-8, -2], [-8, -3], [-9, -3], [-10, -3], [-10, -2], [-11, -2], [-12, -2], [-12, -3], [-12, -4], [-13, -4], [-14, -4], [-14, -5], [-14, -6], [-13, -6], [-12, -6], [-13, -6], [-14, -6], [-13, -6], [-13, -7], [-14, -7], [-15, -7], [-15, -6], [-15, -5], [-16, -5], [-17, -5], [-16, -5], [-16, -6], [-15, -6], [-15, -7], [-15, -8], [-14, -8], [-14, -7], [-14, -6], [-13, -6], [-12, -6], [-11, -6], [-11, -5], [-11, -6], [-11, -7], [-11, -6], [-11, -5], [-11, -4], [-10, -4], [-9, -4], [-9, -3], [-9, -2], [-9, -1], [-9, -2], [-9, -3], [-9, -4], [-9, -3], [-10, -3], [-11, -3], [-12, -3], [-11, -3], [-11, -2], [-11, -3], [-11, -4], [-11, -3], [-11, -4], [-11, -5], [-10, -5], [-11, -5], [-12, -5], [-13, -5], [-12, -5], [-13, -5], [-14, -5], [-15, -5], [-15, -6], [-16, -6], [-17, -6], [-17, -7], [-17, -8], [-17, -9], [-17, -8], [-17, -7], [-16, -7], [-15, -7], [-14, -7], [-14, -8], [-14, -9], [-15, -9], [-16, -9], [-16, -10], [-16, -9], [-16, -8], [-15, -8], [-14, -8], [-15, -8], [-16, -8], [-17, -8], [-16, -8], [-15, -8], [-14, -8], [-15, -8], [-14, -8], [-14, -7], [-14, -6], [-14, -5], [-14, -6], [-14, -7], [-14, -8], [-15, -8], [-16, -8], [-16, -7], [-16, -6], [-17, -6], [-18, -6], [-17, -6], [-17, -5], [-17, -4], [-18, -4], [-19, -4], [-19, -5], [-19, -6], [-19, -7], [-18, -7], [-17, -7], [-17, -6], [-17, -5], [-18, -5], [-19, -5], [-20, -5], [-20, -6], [-19, -6], [-19, -5], [-19, -4], [-18, -4], [-17, -4], [-16, -4], [-16, -5], [-16, -6], [-16, -5], [-16, -4], [-16, -3], [-16, -4], [-15, -4], [-14, -4], [-15, -4], [-16, -4], [-16, -5], [-16, -4], [-16, -3], [-16, -2], [-17, -2], [-17, -3], [-17, -4], [-18, -4], [-17, -4], [-16, -4], [-17, -4], [-16, -4], [-16, -5], [-16, -6], [-16, -7], [-17, -7], [-18, -7], [-19, -7], [-18, -7], [-17, -7], [-18, -7], [-19, -7], [-18, -7], [-17, -7], [-16, -7], [-16, -8], [-16, -9], [-16, -10], [-16, -9], [-17, -9], [-18, -9], [-19, -9], [-19, -10], [-18, -10], [-17, -10], [-18, -10], [-19, -10], [-18, -10], [-18, -11], [-18, -12], [-18, -13], [-17, -13], [-18, -13], [-19, -13], [-18, -13], [-17, -13], [-16, -13], [-16, -12], [-16, -11], [-15, -11], [-14, -11], [-13, -11], [-14, -11], [-15, -11], [-16, -11], [-16, -12], [-16, -13], [-16, -14], [-17, -14], [-18, -14], [-19, -14], [-19, -15], [-19, -14], [-18, -14], [-18, -13], [-18, -12], [-18, -11], [-18, -12], [-18, -11], [-18, -10], [-18, -11], [-18, -12], [-18, -13], [-17, -13], [-16, -13], [-15, -13], [-15, -12], [-16, -12], [-17, -12], [-18, -12], [-18, -13], [-18, -14], [-18, -15], [-18, -14], [-17, -14], [-16, -14], [-16, -15], [-15, -15], [-14, -15], [-13, -15], [-13, -16], [-13, -17], [-12, -17], [-13, -17], [-14, -17], [-14, -18], [-14, -19], [-14,